# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)** 의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)** 을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

### 주요 내용
- Pydantic을 활용해 데이터 추출을 위한 스키마(Schema) 정의  
- LangChain의 Tool Calling 기능을 사용해 LLM이 구조화된 데이터를 반환하도록 설정  
- Few-Shot 프롬프팅(Few-Shot Prompting) 기법을 사용해 성능 향상  
- 다중 엔터티(Multiple Entity) 추출 지원 (여러 개의 인물 정보 추출 가능)

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)** 를 추출하기 위한 예제 **스키마(schema)** 를 정의할 것입니다.

In [ ]:
class Person(BaseModel):

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)** 과 **스키마(schema)** 를 Pydantic으로 문서화  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 합니다.**  
   - 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)** 를 사용하여 **정보 추출기(Information Extractor)** 를 만들어 봅니다.

In [ ]:
# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.

**기능/도구 호출(Function/Tool Calling)** 을 지원하는 모델을 사용해야 합니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [ ]:
class Data(BaseModel):

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)** 를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

## 구조화된 출력 (Structured Outputs)과 함수 호출 (Pydantic 기반)

일반적인 챗봇은 모델이 **자연어로 직접 응답**하지만,  
데이터 분석·API 호출·데이터베이스 저장 같은 상황에서는  
모델이 **정해진 구조(스키마)** 에 맞게 출력을 내보내야 합니다.  
이를 **구조화된 출력(Structured Output)** 이라고 합니다.

---

### 핵심 포인트

- **자연어 응답이 아닌 구조화된 데이터(JSON 형태)** 로 응답.  
- **데이터베이스, API, 파이프라인** 등과의 호환성 보장.  
- **Pydantic 스키마**로 검증된 일관된 데이터 형식 확보.

---

### 구조화된 출력 + 함수 호출 예제 (Pydantic 기반)

1. **Pydantic 스키마 정의**  
   모델이 따라야 할 출력 구조를 정의합니다.
   ```python
   class GeoQuery(BaseModel):
       """위도와 경도 정보를 포함한 위치 데이터"""
       latitude: float = Field(..., description="위도 (−90 ~ 90)")
       longitude: float = Field(..., description="경도 (−180 ~ 180)")


In [ ]:
# 외부 API 호출 함수
def get_temperature(latitude: float, longitude: float) -> float:

In [ ]:
# Pydantic 스키마 정의 (LLM이 이 스키마를 참고해 JSON 생성 -> 파이썬에서 검증/파싱)
class GeoQuery(BaseModel):
# 좌표 추출 (LLM이 JSON 생성 -> Pydantic이 객체화)
# coords는 GeoQuery 인스턴스 (예: GeoQuery(latitude=37.5665, longitude=126.9780))
# 도구(외부 API) 호출
# 최종 응답 생성
# 결과 출력

--------

### 실습 문제: Open-Meteo API를 이용한 실제 날씨 정보 조회 

**목표:**
사용자가 "xx 도시의 날씨를 알려줘"라고 입력하면 모델이 위도/경도를 추론하고, `get_weather(latitude, longitude)` 함수를 호출한 뒤, 결과를 출력하도록 하세요.